In [ ]:
import nltk

In [3]:
messages = [line.rstrip() for line in open('SMSSpamCollection')]

In [10]:
for msg_no, msg in enumerate(messages[:10]):
    print(msg_no, msg)

0 ham	Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
1 ham	Ok lar... Joking wif u oni...
2 spam	Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
3 ham	U dun say so early hor... U c already then say...
4 ham	Nah I don't think he goes to usf, he lives around here though
5 spam	FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, Â£1.50 to rcv
6 ham	Even my brother is not like to speak with me. They treat me like aids patent.
7 ham	As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune
8 spam	WINNER!! As a valued network customer you have been selected to receivea Â£900 prize reward! To claim call 09061701461. Claim code KL341. Valid 1

In [11]:
import pandas as pd

In [12]:
messages = pd.read_csv('SMSSpamCollection',sep='\t',names=['label', 'message'])

In [15]:
messages.groupby('label').describe()

message                                                               
        count unique                                                top freq
label                                                                       
ham      4825   4516                             Sorry, I'll call later   30
spam      747    653  Please call our customer service representativ...    4

In [16]:
messages['length']=messages['message'].apply(len)

In [19]:
from nltk.corpus import stopwords
import string

In [20]:
def text_process(msg):
    nopunc = [char for char in msg if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

In [22]:
messages['message'].head(5).apply(text_process)

0    [Go, jurong, point, crazy, Available, bugis, n...
1                       [Ok, lar, Joking, wif, u, oni]
2    [Free, entry, 2, wkly, comp, win, FA, Cup, fin...
3        [U, dun, say, early, hor, U, c, already, say]
4    [Nah, dont, think, goes, usf, lives, around, t...
Name: message, dtype: object

In [23]:
from sklearn.feature_extraction.text import CountVectorizer

In [24]:
bow_transformer = CountVectorizer(analyzer=text_process).fit(messages['message'])

In [54]:
print(bow_transformer.get_feature_names()[11424])

…Thanks


In [47]:
messages_bow = bow_transformer.transform(messages['message'])

In [50]:
sparsity = (100.0 * messages_bow.nnz)/(messages_bow.shape[0] * messages_bow.shape[1])
print('sparsity: {}'.format(sparsity))

sparsity: 0.07940295412668218


In [51]:
from sklearn.feature_extraction.text import TfidfTransformer

In [56]:
tfidf_transformer = TfidfTransformer().fit(messages_bow)

In [58]:
messages_tfidf = tfidf_transformer.transform(messages_bow)

In [59]:
 from sklearn.naive_bayes import MultinomialNB

In [60]:
spam_detect_model = MultinomialNB().fit(messages_tfidf, messages['label'])

In [62]:
all_pred = spam_detect_model.predict(messages_tfidf)

In [63]:
from sklearn.model_selection import train_test_split

In [64]:
msg_train, msg_test, label_train, label_test = train_test_split(messages['message'], messages['label'], test_size=0.3)

In [65]:
from sklearn.pipeline import Pipeline

In [66]:
pipeline = Pipeline([
    ('bow', CountVectorizer(analyzer=text_process)),
    ('tfidf', TfidfTransformer()),
    ('classifier', MultinomialNB())
])

In [67]:
pipeline.fit(msg_train, label_train)

Pipeline(steps=[('bow',
                 CountVectorizer(analyzer=<function text_process at 0x000002071CFC11F8>)),
                ('tfidf', TfidfTransformer()),
                ('classifier', MultinomialNB())])

In [68]:
pred = pipeline.predict(msg_test)

In [69]:
from sklearn.metrics import classification_report, confusion_matrix

In [70]:
print(confusion_matrix(label_test, pred))
print(classification_report(label_test, pred))

[[1474    0]
 [  60  138]]
              precision    recall  f1-score   support

         ham       0.96      1.00      0.98      1474
        spam       1.00      0.70      0.82       198

    accuracy                           0.96      1672
   macro avg       0.98      0.85      0.90      1672
weighted avg       0.97      0.96      0.96      1672

